In [1]:
import pandas as pd
import numpy as np
import glob
import os
import re

In [2]:
def clean_df():
    path = f'{file_dir}/Resources/first_round_csvs'
    all_files = glob.glob(os.path.join(path + "/*.csv"))

    li = []

    for filename in all_files:
        df = pd.read_csv(filename,encoding_errors='ignore', index_col=None, header=0)
        li.append(df)

    first_round_df = pd.concat(li, axis=0, ignore_index=True)
    first_round_df = first_round_df.sort_values(by=['Year', 'RdPck']).reset_index()
    first_round_df = first_round_df.drop(['index', 'DT', 'FrRnd', 'OvPck'], axis=1)
    
    first_round_df['Tm'] = first_round_df['Tm'].str.replace(r'via(.*$)', '', regex=True)
    first_round_df['Name'] = first_round_df['Name'].str.replace(r'\((.+?)\)', '', regex=True) 
    first_round_df['Type'] = first_round_df['Type'].fillna('None')
    first_round_df['Type'] = first_round_df['Type'].fillna('None')
    
    first_round_df['WAR'] = first_round_df['WAR'].fillna(0)
    first_round_df['G'] = first_round_df['G'].fillna(0)
    first_round_df['AB'] = first_round_df['AB'].fillna(0)
    first_round_df['HR'] = first_round_df['HR'].fillna(0)
    first_round_df['BA'] = first_round_df['BA'].fillna(0)
    first_round_df['OPS'] = first_round_df['OPS'].fillna(0)
    first_round_df['G.1'] = first_round_df['G.1'].fillna(0)
    first_round_df['W'] = first_round_df['W'].fillna(0)
    first_round_df['L'] = first_round_df['L'].fillna(0)
    first_round_df['ERA'] = first_round_df['ERA'].fillna(0)
    first_round_df['WHIP'] = first_round_df['WHIP'].fillna(0)
    first_round_df['SV'] = first_round_df['SV'].fillna(0)
    
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return 0
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
            s = re.sub('\$|,','', s)
            value = float(s)
            return value
        else:
            return 0
    
    def parse_rnd(s):
        if type(s) != str:
            return int(s)
        elif re.match('1s', s):
            s = re.sub('s', '', s)
            value = int(s)
            return value
        else:
            return int(s)    
    
    first_round_df['Bonus'] = first_round_df['Bonus'].apply(parse_dollars)
    first_round_df['Rnd'] = first_round_df['Rnd'].apply(parse_rnd)
    
    return first_round_df

In [3]:
file_dir = '/Users/daniellesilvaggio/Desktop/UCSD/Final_Project_Baseball_draft_prediction'
player_file = f'{file_dir}/Resources/first_round_csvs'

In [4]:
player_file = clean_df()
first_round_df = player_file
first_round_df.head()

,Year,Rnd,RdPck,Tm,Signed,Bonus,Name,Pos,WAR,G,...,OPS,G.1,W,L,ERA,WHIP,SV,Type,Drafted Out of,Team Made Playoffs
0,2000,1,1,Marlins,Y,3000000.0,Adrian Gonzalez,1B,43.5,1929.0,...,0.843,0.0,0.0,0.0,0.00,0.00,0.0,HS,"Eastlake HS (Chula Vista, CA)",0
1,2000,1,2,Twins,Y,2500000.0,Adam Johnson,RHP,-1.1,1.0,...,0.000,9.0,1.0,3.0,10.25,2.05,0.0,4Yr,"California State University, Fullerton (Fuller...",0
2,2000,1,3,Cubs,Y,2750000.0,Luis Montanez,SS,-1.3,129.0,...,0.586,0.0,0.0,0.0,0.00,0.00,0.0,HS,"Coral Park HS (Miami, FL)",0
3,2000,1,4,Royals,Y,2500000.0,Mike Stodolka,LHP,0.0,0.0,...,0.000,0.0,0.0,0.0,0.00,0.00,0.0,HS,"Centennial HS (Corona, CA)",0
4,2000,1,5,Expos,Y,2950000.0,Justin Wayne,RHP,-1.5,26.0,...,0.000,26.0,5.0,8.0,6.13,1.65,0.0,4Yr,"Stanford University (Palo Alto, CA)",0


In [5]:
first_round_df.dtypes

Year                    int64
Rnd                     int64
RdPck                   int64
Tm                     object
Signed                 object
Bonus                 float64
Name                   object
Pos                    object
WAR                   float64
G                     float64
AB                    float64
HR                    float64
BA                    float64
OPS                   float64
G.1                   float64
W                     float64
L                     float64
ERA                   float64
WHIP                  float64
SV                    float64
Type                   object
Drafted Out of         object
Team Made Playoffs      int64
dtype: object

In [6]:
random_forest_df = first_round_df.drop(['Name','Drafted Out of', 'Rnd'], 1)
random_forest_df

/Users/daniellesilvaggio/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Year,RdPck,Tm,Signed,Bonus,Pos,WAR,G,AB,HR,BA,OPS,G.1,W,L,ERA,WHIP,SV,Type,Team Made Playoffs
0,2000,1,Marlins,Y,3000000.0,1B,43.5,1929.0,7139.0,317.0,0.287,0.843,0.0,0.0,0.0,0.00,0.00,0.0,HS,0
1,2000,2,Twins,Y,2500000.0,RHP,-1.1,1.0,2.0,0.0,0.000,0.000,9.0,1.0,3.0,10.25,2.05,0.0,4Yr,0
2,2000,3,Cubs,Y,2750000.0,SS,-1.3,129.0,305.0,5.0,0.223,0.586,0.0,0.0,0.0,0.00,0.00,0.0,HS,0
3,2000,4,Royals,Y,2500000.0,LHP,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.00,0.00,0.0,HS,0
4,2000,5,Expos,Y,2950000.0,RHP,-1.5,26.0,12.0,0.0,0.000,0.000,26.0,5.0,8.0,6.13,1.65,0.0,4Yr,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,2015,38,Rockies,Y,2000000.0,3B,0.0,48.0,138.0,3.0,0.196,0.615,0.0,0.0,0.0,0.00,0.00,0.0,HS,0
742,2015,39,Cardinals,Y,1800000.0,RHP,0.2,25.0,18.0,0.0,0.111,0.222,49.0,5.0,4.0,4.12,1.30,0.0,HS,1
743,2015,40,Brewers,Y,1250000.0,LHP,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.00,0.00,0.0,4Yr,0
744,2015,41,Braves,Y,1600000.0,3B,7.8,370.0,1360.0,79.0,0.270,0.834,0.0,0.0,0.0,0.00,0.00,0.0,HS,0


In [7]:
random_forest_df.count()

Year                  746
RdPck                 746
Tm                    746
Signed                746
Bonus                 746
Pos                   746
WAR                   746
G                     746
AB                    746
HR                    746
BA                    746
OPS                   746
G.1                   746
W                     746
L                     746
ERA                   746
WHIP                  746
SV                    746
Type                  746
Team Made Playoffs    746
dtype: int64

# Random Forest Model

In [8]:
# Generate our categorical variable lists
random_cat = random_forest_df.dtypes[random_forest_df.dtypes == "object"].index.tolist()

In [9]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(random_forest_df[random_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(random_cat)
encode_df.head()

/Users/daniellesilvaggio/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Tm_Angels,Tm_Angels,Tm_Astros,Tm_Astros,Tm_Athletics,Tm_Athletics,Tm_Blue Jays,Tm_Blue Jays,Tm_Braves,Tm_Braves,...,Pos_LHP,Pos_OF,Pos_RF,Pos_RHP,Pos_SS,Pos_lHP,Type_4Yr,Type_HS,Type_JC,Type_None
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
random_forest_df = random_forest_df.merge(encode_df,left_index=True, right_index=True)
random_forest_df = random_forest_df.drop(random_cat,1)
random_forest_df.head()

/Users/daniellesilvaggio/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Year,RdPck,Bonus,WAR,G,AB,HR,BA,OPS,G.1,...,Pos_LHP,Pos_OF,Pos_RF,Pos_RHP,Pos_SS,Pos_lHP,Type_4Yr,Type_HS,Type_JC,Type_None
0,2000,1,3000000.0,43.5,1929.0,7139.0,317.0,0.287,0.843,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2000,2,2500000.0,-1.1,1.0,2.0,0.0,0.000,0.000,9.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2000,3,2750000.0,-1.3,129.0,305.0,5.0,0.223,0.586,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,2000,4,2500000.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2000,5,2950000.0,-1.5,26.0,12.0,0.0,0.000,0.000,26.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [11]:
# Define the features set.
X = random_forest_df.copy()
X = X.drop("Team Made Playoffs", axis=1)
X.head()

,Year,RdPck,Bonus,WAR,G,AB,HR,BA,OPS,G.1,...,Pos_LHP,Pos_OF,Pos_RF,Pos_RHP,Pos_SS,Pos_lHP,Type_4Yr,Type_HS,Type_JC,Type_None
0,2000,1,3000000.0,43.5,1929.0,7139.0,317.0,0.287,0.843,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2000,2,2500000.0,-1.1,1.0,2.0,0.0,0.000,0.000,9.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2000,3,2750000.0,-1.3,129.0,305.0,5.0,0.223,0.586,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,2000,4,2500000.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2000,5,2950000.0,-1.5,26.0,12.0,0.0,0.000,0.000,26.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
y = random_forest_df["Team Made Playoffs"].ravel()
y[:5]

array([0, 0, 0, 0, 0])

In [13]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Creating a StandardScaler instance.
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [15]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0])

In [16]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,120,10
Actual 1,46,11


In [17]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [18]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,120,10
Actual 1,46,11


Accuracy Score : 0.7005347593582888
Classification Report
              precision    recall  f1-score   support

           0       0.72      0.92      0.81       130
           1       0.52      0.19      0.28        57

    accuracy                           0.70       187
   macro avg       0.62      0.56      0.55       187
weighted avg       0.66      0.70      0.65       187



In [19]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.09209988192940001, 'RdPck'),
 (0.0855645105305764, 'Bonus'),
 (0.07276630565056115, 'Year'),
 (0.04929317895005376, 'WAR'),
 (0.04447096947679634, 'G'),
 (0.043846954421098344, 'AB'),
 (0.04174698769818195, 'OPS'),
 (0.03541413640241195, 'BA'),
 (0.027673321707922926, 'HR'),
 (0.027580599981500274, 'G.1'),
 (0.027479285089780984, 'Tm_Yankees'),
 (0.027386631122374907, 'WHIP'),
 (0.025956256995441764, 'ERA'),
 (0.02399529560061553, 'Tm_Cardinals'),
 (0.022459670565856044, 'W'),
 (0.02203043882319845, 'L'),
 (0.015829926916553327, 'Tm_Rays'),
 (0.01527497753039345, 'SV'),
 (0.012789143323583338, 'Type_HS'),
 (0.012066307299163673, 'Tm_Braves'),
 (0.011916749221240438, 'Type_4Yr'),
 (0.010993161703818153, 'Pos_OF'),
 (0.010603965083095591, 'Pos_RHP'),
 (0.01022811108264573, 'Pos_SS'),
 (0.010136530914505136, 'Tm_Blue Jays'),
 (0.00981351373854889, 'Tm_Athletics'),
 (0.00958218061681063, 'Tm_Dodgers'),
 (0.00951828702923, 'Tm_Red Sox'),
 (0.008895714007853016, 'Tm_Padres'),
 (0.0087072

# Deep Learning Model

In [20]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=100,
        step=2), activation=activation))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=100,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [21]:
# Import the kerastuner library
import kerastuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


/Users/daniellesilvaggio/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


2022-07-26 14:02:47.892794: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

INFO:tensorflow:Oracle triggered exit


In [23]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 83,
 'num_layers': 4,
 'units_0': 99,
 'units_1': 47,
 'units_2': 43,
 'units_3': 91,
 'units_4': 51,
 'units_5': 67,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0044'}

In [24]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6/6 - 0s - loss: 0.5801 - accuracy: 0.7701 - 428ms/epoch - 71ms/step
Loss: 0.5801041722297668, Accuracy: 0.7700534462928772
